In [4]:
%pip install geopy

     -------------------------------------- 119.8/119.8 KB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 40.3/40.3 KB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Kashif Khan\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
## Load libraries
import pandas as pd
import numpy as np
import os
import time
import glob
import re
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.io as sio
# import pywt
import scipy.stats as stats
import datetime as dt
import itertools
import folium
from geopy import distance
import csv
from sklearn.cluster import KMeans
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
%matplotlib inline

In [2]:
## Setup working directory and data directory
# DIR = ''
# os.chdir(DIR)
DATA_DIR = 'E:\\MAHE\\1st Sem\\Projects\\Dangerous Road Curvature\\'

In [4]:
# Read data
filename = DATA_DIR + 'input1.csv'
df_full = pd.read_csv(filename, skiprows = 0, sep = ',', index_col = False)            
df_full.columns

Index(['Unnamed: 0', 'time', 'Accr_x', 'Accr_y', 'Accr_z', 'Rot_x', 'Rot_z',
       'Magnitude_Accelerometer', 'Magnitude_Gyroscope', 'Latitude',
       'Longitude'],
      dtype='object')

In [5]:
## Wrangle dataframe
# Specify columns that will be selected
features = ['Accr_x',
            'Accr_y',
            'Accr_z',
            'Magnitude_Accelerometer',
            'Magnitude_Gyroscope',
            'Latitude',
            'Longitude',
            'time'
            ]
df = df_full.loc[:, df_full.columns.isin(features)]
df.dropna(inplace = True)

## Rename the selected columns
df.columns = ['t', 'a_x', 'a_y', 'a_z', 'Mag_Accr', 'Mag_Gyro', 'lat', 'lon']

## Adjust start time to 0
df = df.assign(t = df['t'] - df['t'].values[0]) 
df.head()

C:\Users\Kashif Khan\AppData\Local\Temp\ipykernel_14028\375623085.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace = True)


,t,a_x,a_y,a_z,Mag_Accr,Mag_Gyro,lat,lon
0,0.000,0.989624,0.071533,0.253296,1.024027,10.930066,13.353532,74.790861
1,0.022,0.956299,0.061035,0.211060,0.981213,13.416910,13.353532,74.790861
2,0.023,0.970093,0.041992,0.207031,0.992827,11.719386,13.353532,74.790861
3,0.025,0.931030,0.057251,0.220337,0.958459,9.644327,13.353532,74.790861
4,0.026,0.846558,0.126343,0.230347,0.886387,10.297591,13.353532,74.790861


In [34]:
## Write segmented output to CSV file
segment_length = 100 # set a segment length of 100 metres
data = []
segment_filename = DATA_DIR + 'segmented_output1.csv'
startindex = 0
j = 0
coords_1 = (df.loc[startindex,'lat'], df.loc[startindex,'lon'])
for index in range(len(df)):    
  subdata = []
  subdata.append(['Road segment', j])
  subdata.append(['Start latitude and longitude', coords_1[0], coords_1[1]])
  coords_2 = (df.loc[index,'lat'], df.loc[index,'lon'])   
  d = distance.distance(coords_1, coords_2).m 
  if d >= segment_length:    
    subdata.append(['End latitude and longitude', coords_2[0], coords_2[1]])
    subdata.append(['Distance', d])
    subdata.append(['Time']+list(df.loc[startindex:index+1, 't'].values-df.loc[startindex, 't']))
    # subdata.append(['X_Acceleration']+list(df.loc[startindex:index+1, 'a_x'].values))
    # subdata.append(['Y_Acceleration']+list(df.loc[startindex:index+1, 'a_y'].values))
    subdata.append(['Z_Acceleration']+list(df.loc[startindex:index+1, 'a_z'].values))

    data.append(subdata) 
    if j == 0:
      writemode = "w"
    else:
      writemode = "a"   
    with open(segment_filename, writemode, newline = "") as f:
      writer = csv.writer(f)
      writer.writerows(subdata)
    startindex = index
    j = j+1
    coords_1 = (df.loc[startindex,'lat'], df.loc[startindex,'lon'])

In [11]:
segment_length = 100 # set a segment length of 100 metres
data = []
segment_filename = DATA_DIR + 'segmented_output1.csv'
startindex = 0
j = 0
json_li = []
coords_1 = (df.loc[startindex,'lat'], df.loc[startindex,'lon'])
for index in range(len(df)):    
    subdata = []
    subdata.append(['Road segment', j])
    subdata.append(['Start latitude and longitude', coords_1[0], coords_1[1]])
    coords_2 = (df.loc[index,'lat'], df.loc[index,'lon'])   
    d = distance.distance(coords_1, coords_2).m 
    if d >= segment_length:
        json_data = {}    
        subdata.append(['End latitude and longitude', coords_2[0], coords_2[1]])
        subdata.append(['Distance', d])
        subdata.append(['Time']+list(df.loc[startindex:index+1, 't'].values-df.loc[startindex, 't']))
        subdata.append(['X_Acceleration']+list(df.loc[startindex:index+1, 'a_x'].values))
        subdata.append(['Y_Acceleration']+list(df.loc[startindex:index+1, 'a_y'].values))
        subdata.append(['Z_Acceleration']+list(df.loc[startindex:index+1, 'a_z'].values))
        subdata.append(['Magnitude_Accelerometer']+list(df.loc[startindex:index+1, 'Mag_Accr'].values))
        subdata.append(['Magnitude_Gyroscope']+list(df.loc[startindex:index+1, 'Mag_Gyro'].values))

        json_data[subdata[0][0]] = subdata[0][1]    # Road Segment
        json_data[subdata[1][0]] = subdata[1][1:]   # Start Latitude and Longitude
        json_data[subdata[2][0]] = subdata[2][1:]   # End Latitude and Longitude
        json_data[subdata[3][0]] = subdata[3][1:]   # Distance
        json_data[subdata[4][0]] = subdata[4][1:]   # Time
        json_data[subdata[5][0]] = subdata[5][1:]   # X_Acceleration
        json_data[subdata[6][0]] = subdata[6][1:]   # Y_Acceleration
        json_data[subdata[7][0]] = subdata[7][1:]   # Z_Acceleration
        json_data[subdata[8][0]] = subdata[8][1:]   # Magnitude_Accelerometer
        json_data[subdata[9][0]] = subdata[9][1:]   # Magnitude_Gyroscope

        json_li.append(json_data)

        print(subdata[0][0])
        # data.append(subdata) 
        
        # if j == 0:
        #     writemode = "w"
        # else:
        #     writemode = "a"   
        # with open(segment_filename, writemode, newline = "") as f:
        #     writer = csv.writer(f)
        #     writer.writerows(subdata)
        startindex = index
        j = j+1
        coords_1 = (df.loc[startindex,'lat'], df.loc[startindex,'lon'])
# print(json_li)

Road segment
Road segment
Road segment
Road segment
Road segment
Road segment
Road segment
Road segment
Road segment
Road segment
Road segment
Road segment
Road segment
Road segment
Road segment
Road segment


In [12]:
import json
json_object = json.dumps(json_li)
with open('data1.json', 'w') as f:
    json.dump(json_object, f)

In [17]:
json_df = pd.DataFrame(json_li)

In [19]:
json_df.to_csv('csv_file.csv')

In [20]:
json_df.to_json('json_file.json')